In [91]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

In [92]:
df = pd.read_csv('Groceries_dataset.csv')
df.head()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [93]:
df.shape[0]

38765

In [94]:
df.nunique()

Member_number      3898
Date                728
itemDescription     167
dtype: int64

In [95]:
newDf = pd.DataFrame(df.groupby('Date').nunique().index)
newDf['Member_Count'] = df.groupby('Date')['Member_number'].nunique().values
newDf['Item_Count'] = df.groupby('Date')['itemDescription'].nunique().values
newDf['Items'] = df.groupby('Date')['itemDescription'].unique().values
newDf.set_index('Date', inplace=True)

In [96]:
## Now we will make new dataframe with unique id as date and
newDf

,Member_Count,Item_Count,Items
Date,,,
01-01-2014,21,30,"[cleaner, sausage, tropical fruit, whole milk,..."
01-01-2015,16,34,"[canned beer, frozen meals, butter, ham, citru..."
01-02-2014,29,45,"[other vegetables, pip fruit, frankfurter, sug..."
01-02-2015,20,31,"[whole milk, canned beer, bottled water, soda,..."
01-03-2014,24,36,"[sausage, meat, sugar, bottled beer, whole mil..."
...,...,...,...
31-07-2015,24,35,"[vinegar, chicken, root vegetables, sausage, w..."
31-08-2014,21,32,"[pork, other vegetables, beef, whole milk, hyg..."
31-08-2015,15,32,"[brown bread, frankfurter, chicken, citrus fru..."


In [97]:
## Now making transaction list
transactions = list()
for items in newDf['Items']:
    transactions.append(items)

In [98]:
## Now using apriori algorithm
## Now training the transaction dataset on apriori model
from apyori import apriori
rules = apriori(transactions=transactions, min_support = 0.003, min_confidence = 0.2, min_lift = 3, min_length = 2, max_length = 2)
## Here min_support is calculated by considering product is bought atleast 3 times a day => 3*7/7051

In [99]:
## Now visualising
results = list(rules)
results

[RelationRecord(items=frozenset({'Instant food products', 'liqueur'}), support=0.004120879120879121, ordered_statistics=[OrderedStatistic(items_base=frozenset({'liqueur'}), items_add=frozenset({'Instant food products'}), confidence=0.33333333333333337, lift=4.257309941520468)]),
 RelationRecord(items=frozenset({'baking powder', 'decalcifier'}), support=0.008241758241758242, ordered_statistics=[OrderedStatistic(items_base=frozenset({'decalcifier'}), items_add=frozenset({'baking powder'}), confidence=0.6666666666666667, lift=4.412121212121213)]),
 RelationRecord(items=frozenset({'candles', 'whisky'}), support=0.004120879120879121, ordered_statistics=[OrderedStatistic(items_base=frozenset({'whisky'}), items_add=frozenset({'candles'}), confidence=0.375, lift=4.475409836065574)]),
 RelationRecord(items=frozenset({'cereals', 'cookware'}), support=0.005494505494505495, ordered_statistics=[OrderedStatistic(items_base=frozenset({'cookware'}), items_add=frozenset({'cereals'}), confidence=0.23529

In [100]:
## Now putting the result into pandas dataframe
def inspect(results):
    lhs         = [tuple(result[2][0][0])[0] for result in results]
    # Here first we have accessed the index 2 of result and then the 0th index and then 0th index and then inside it
    rhs         = [tuple(result[2][0][1])[0] for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(lhs, rhs, supports, confidences, lifts))
resultsinDataFrame = pd.DataFrame(inspect(results), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])



In [101]:
resultsinDataFrame.sort_values(by='Lift', ascending=False)

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
21,liqueur,rum,0.004121,0.333333,8.088889
16,frozen fruits,light bulbs,0.004121,0.272727,7.090909
29,rubbing alcohol,turkey,0.004121,0.600000,5.983562
10,honey,female sanitary products,0.005495,0.307692,5.600000
4,cooking chocolate,cleaner,0.004121,0.200000,5.020690
31,specialty vegetables,softener,0.004121,0.272727,4.842572
11,prosecco,female sanitary products,0.006868,0.263158,4.789474
12,whisky,finished products,0.004121,0.375000,4.789474
18,hair spray,vinegar,0.004121,0.333333,4.758170
15,rubbing alcohol,frozen fish,0.004121,0.600000,4.646809
